<!--NAVIGATION-->
< [Co-occurrence analysis](tutorial_4.ipynb) | [Contents](index.ipynb) | [Model application and visualization](tutorial_6.ipynb) >

# Model inference

In this tutorial the application and the basic operations of the topic modelling packages in R are introduced. We apply the LDA model from [1] to a publicly available dataset: the Job postings from the Kaggle page. 

We introduce different observations which can be derived from the posterior distributions of the LDA model. These basic observations allow to interpret and quantify semantically coherent topics within the data. It is also possible to compare the topical composition of a text source with other sources.

First, we start with initializing our text data set, creating a quanteda-corpus object from it, apply some pre-processing measures, and finally create a Document-Term-Matrix (DTM). For the model inferences we use the `topicmodels`-package of R [2].

In [ ]:
# Setup and initial package import
options(stringsAsFactors = FALSE)
require(quanteda)
require(magrittr)
require(dplyr)
require(topicmodels)
require(lda)
require(openNLP)
require(NLP)
require(data.table)
# Read stop word file
english_stopwords <- readLines("resources/stopwords_en.txt", encoding = "UTF-8")

We read a CSV containing 19.000 "Job Description". The texts are freely available from https://www.kaggle.com/madhab/jobposts. Our CSV file has the format: `"jobpost";"date";"Title"; etc.`..

For linguistic preprocessing to unify the vocabulary, we can use either stemming or lemmatization. Stemming can be done by rules for a given language (use the `stemDocument` function). We will use a simple lemmatization on the basis of a lookup table (`resources/baseform_en.tsv`).

In [ ]:
# Read text data from CSV file
textdata <- read.csv("data/data job posts.csv", header = TRUE, sep = ",", encoding = "UTF-8",quote = "\"")
textdata <- as.data.table(textdata)

english_stopwords <- readLines("resources/stopwords_en.txt", encoding = "UTF-8")

textdata %<>% filter(!duplicated(jobpost))
textdata %<>% mutate(d_id = 1:nrow(textdata))

#Build a dictionary of lemmas
lemmaData <- read.csv2("resources/baseform_en.tsv", sep="\t", header=FALSE, encoding = "UTF-8", stringsAsFactors = F)

data_corpus <- corpus(textdata$jobpost, docnames = textdata$d_id)
 
data_dfm_entries <- data_corpus %>% tokens() %>% tokens_remove(pattern = c(stopwords(),english_stopwords)) %>%
  tokens(remove_punct = TRUE, remove_numbers = TRUE, remove_symbols = TRUE) %>% tokens_tolower() %>% 
  tokens_replace(., lemmaData$V1, lemmaData$V2) %>%
  tokens_ngrams(1) %>% dfm() 


data_dfm_entries_sub <- data_dfm_entries %>% dfm_trim(min_docfreq = 5) %>%
  dfm_select(pattern = "[a-z]", valuetype = "regex", selection = 'keep')

colnames(data_dfm_entries_sub) <- colnames(data_dfm_entries_sub) %>% stringi::stri_replace_all_regex("[^_a-z]", "") 

data_dfm_entries_sub <- dfm_compress(data_dfm_entries_sub, "features")

data_dfm_entries_sub <- 
  data_dfm_entries_sub[rowSums(data_dfm_entries_sub) >=10, nchar(colnames(data_dfm_entries_sub)) > 1]


Now we save the corpus objects for later use and create the DTM. We keep only terms occurring more than 5 times.

In [ ]:
save(data_corpus, file = "corpus.RData")

# RENAME
DTM <- data_dfm_entries_sub

We save our DTM for later use in other tutorials, so we do not need to compute it all over again.

In [ ]:
# have a look at the number of documents and terms in the matrix
dim(DTM)
save(DTM, file = "DTM.RData")

## LDA Model inference

The inference of topic models aims for the separation of a document collection into a fixed set of topics. Topic models are unsupervised machine learning algorithms. Thus, they are suited best for exploring data. It is useful to experiment with different parameters to find the optimal setting for the intended analysis.

For parametrized models like the Latent Dirichlet Allocation the number of topics K has to be set in advance. The decision for an optimal K is dependent on different factors. If K is too low the collection of documents is separated into few very coarse semantic coherent topics. If K is too large the result is a separation into very detailed topics which are hard to distinguish or interpret. 

As a start we choose a manually overseeable number of 20 topics. The other hyper-parameters relevant for the inference process are set to `alpha = 20` as prior for the topic-distributions, and beta to an automatically estimated value based on the given data. The inference is run with Gibbs sampling for 500 iterations.

In [ ]:
# number of topics
K <- 20

selector <- textdata %>% filter(Company != "Samsung Electronics Representative Office in Armenia") %>% select("d_id")
selector <- rownames(DTM) %in% as.character(selector$d_id)

# compute the LDA model, inference via 500 iterations of Gibbs sampling
topicModel <- LDA(DTM[selector,], K, method="Gibbs", control=list(iter = 500, verbose = 20, alpha = 0.2, estimate.beta = TRUE))
save(topicModel, file = "topicModel.RData")

The inference can take quite long, dependent on the size of the vocabulary, the number of documents and the setting of K. Our calculation may take a few minutes. If an inference process takes too long on your computer, you can reduce the size of the vocabulary by raising the minimum frequency passed to the creation process of the document term matrix or reduce the number of iterations.

After running the inference process for a while, two posterior distributions can be calculated. The first, called `theta`, represents the distributions of all topics K w.r.t. each document. The second posterior parameter, called `beta` represents the distribution of all terms V w.r.t. each topic. V represents the length of the vocabulary (`r ncol(DTM)`). Let's take a closer look:

In [ ]:
# have a look to some of the results (posterior distributions)
tmResult <- posterior(topicModel)
save(tmResult, file = "tmResult.RData") #we save the posterior for later use
# format of the resulting object
attributes(tmResult)

ncol(DTM)              # lengthOfVocab
# topics are probability distributions over the entire vocabulary
beta <- tmResult$terms   # get beta from results
dim(beta)                # K distributions over nTerms(DTM) terms
rowSums(beta)            # rows in beta sum to 1
nrow(DTM)               # size of collection 
# for every document we have a probaility distribution of its contained topics
theta <- tmResult$topics 
dim(theta)               # nDocs(DTM) distributions over K topics
rowSums(theta)[1:10]     # rows in theta sum to 1

## Ranking of words

Now let us look at the 10 **most probable words** for each topic within the inferred `beta` distribution.

In [ ]:
terms(topicModel, 10)

Instead of numbers we would like to set descriptive names for the topics in the next step. For this reason, we concatenate the top 5 probable words of each topic into a pseudo-name for each topic. 

In [ ]:
# the function terms extracts the most likely terms for each topic
top5termsPerTopicProb <- terms(topicModel, 5)
topicNames <- apply(top5termsPerTopicProb, 2, paste, collapse=" ")

Often, the ranking by probability returns words which might be very frequent in a topic but, at the same time, are not very distinctive. This might be the case for high frequent words such as "year", "job" or "qualification" that are no stopwords, but relevant for many topics. An alternative to probability based ranking is the ranking of the words by a tf/idf-like ranking mechanism. Within this procedure words get punished if they appear in many topics with high probability.

### Ranking by score (lda package)

This mechanism ranks the words according to a score defined by [3] and is defined as 

$$relevance(w|t) = p(w, t)  \cdot \left(\log p(w|t) - \frac{1}{K} \sum_{t'} \log p(w|t')\right).$$

In [ ]:
# LDA package function top.topic.words needs a K x V matrix
dim(tmResult$terms) # The output from the topicmodels package has the right dimension

# The dimension is compatible so we can put beta directly into the function
top5termsPerTopicScore <- top.topic.words(beta, num.words = 5, by.score = T)
topicNamesByScore <- apply(top5termsPerTopicScore, 2, paste, collapse = " ")
print(topicNamesByScore)

Now, for instance the term "year", which occurred `r sum(unlist(top5termsPerTopicProb) == "year")` time among the top 5 terms ranked by probability, only occurs `r sum(unlist(top5termsPerTopicScore) == "year")` time(s) using the score ranking.

### Ranking by score II (LDAvis package)

This alternate scoring strategy ranks the words according to a score defined by [4] and is defined as

$$relevance(w|t) = \lambda  \cdot \log p(w|t) + (1 - \lambda)\cdot \log \frac{p(w|t)}{p(w)}.$$

The parameter $\lambda$, ranging between 0 and 1, influences the scoring by penalizing if a term is not only likely to a specific topic, but also to the entire collection. $\lamba = 1$ simply ranks a term according to its probability. $\lamba$ near 0 puts emphasis on the exlusivity of a term for a topic. 


In [ ]:
# Unfortunately, there is no package containing this formula. 
# So, we have to implement it on our own
scoreByLambda <- function(p_w_t, num.words = 5, p_w, lambda)
{
  apply(p_w_t, 1, function(x, num.words, p_w,lambda) {
    x <- lambda * log(x) + (1 - lambda) * log(x / p_w)
    return(names(sort(x, decreasing = T)[1:num.words]))
  }, num.words, p_w, lambda)
  
}

p_w <- slam::col_sums(DTM) / sum(DTM)
top5termsPerTopicScoreII <- scoreByLambda(tmResult$terms, num.words = 5, p_w, lambda = 0.6)
topicNamesByScoreII <- apply(top5termsPerTopicScoreII, 2, paste, collapse = " ")
print(top5termsPerTopicScoreII)


@sievert_ldavis:_2014 claim $\lambda = 0.6$ as a good value balancing between the two terms of the scoring formula.

## Ranking of topics

Which topics are prominent in a document collection? Multiple approaches exist to determine a ranking of the inferred topics.

### Approach 1

We sort the topics by their probability among the whole collection:


In [ ]:
# What are the most probable topics in the entire collection?
topicProportions <- colSums(theta) / sum(theta)  # mean probablities over all documents
names(topicProportions) <- topicNamesByScoreII     # assign the topic names we created before
sort(topicProportions, decreasing = TRUE) # show summed proportions in decreased order

Some topics with a higher probability are observable. Those topics describe rather general contexts. Other topics represent tangible and interpretable topics (Cuba island, slavery, ...). 

## Approach 2

We count how often a topic appears as the primary topic within the documents. This method is called *Rank-1*.

In [ ]:
countsOfPrimaryTopics <- rep(0, K)
names(countsOfPrimaryTopics) <- topicNamesByScoreII
for (i in 1:dim(theta)[1]) {
  topicsPerDoc <- theta[i, ] # select topic distribution for document i
  # get first element position from ordered list
  primaryTopic <- order(topicsPerDoc, decreasing = TRUE)[1] 
  countsOfPrimaryTopics[primaryTopic] <- countsOfPrimaryTopics[primaryTopic] + 1
}
sort(countsOfPrimaryTopics, decreasing = TRUE)

As one can see, the Rank-1 sorting puts some very specific topics more to the upper ranks (e.g. terrorism). This sorting can be used for further analysis steps like the semantic interpretation of the topics in the documents, time series analysis of topics or the filtering of the documents w.r.t. specific topics. 

## Predict topics on new data

Once a topic model has been created we can use the inferred posterior to infer a unknown document collection w.r.t. our reference topic model. This can very helpful in case topics should be compared with respect to a reference collection. In antother scenario, where a very large corpus should be modeled, we could calculate a model on a smaller sample of documents and then use this model to infer topics for the remaining out-of-sample documents.

IMPORTANT: The matrix for the out-of-sample documents must be of the same dimension that the matrix from which the model was created. This could lead to empty or very short documents due to the unknown vocabulary of new documents


In [ ]:
selector <- textdata %>% filter(Company == "Samsung Electronics Representative Office in Armenia") %>% select("d_id")
selector <- rownames(DTM) %in% as.character(selector$d_id)

new_data <- posterior(topicModel, DTM[selector, ])
save(new_data, file = "tmResult_documents.RData")

# Proportions of topics
topicProportionsNewData <- colSums(new_data$topics) / sum(new_data$topics)
names(topicProportionsNewData) <- topicNamesByScoreII 
sort(topicProportionsNewData, decreasing = TRUE)


# Rank-1 metric
countsOfPrimaryTopicsNewData <- rep(0, K)
names(countsOfPrimaryTopicsNewData) <- topicNamesByScore
for (i in 1:nrow(DTM[selector, ])) {
  topicsPerDoc <- new_data$topics[i, ] 
  primaryTopic <- order(topicsPerDoc, decreasing = TRUE)[1] 
  countsOfPrimaryTopicsNewData[primaryTopic] <- countsOfPrimaryTopicsNewData[primaryTopic] + 1
}
sort(countsOfPrimaryTopicsNewData, decreasing = TRUE)

## Use POS-Tagging for pre-processing

Very frequent words can impede the modelling itself, resp. the interpretation of a topic model result. Due to this, filtering stop words beforhand is usually a very good idea. 

Further, it can be a good strategy to filter specific word types such as verbs or adjectives. In the next code block we'll process the corpus with POS-tagging and only use nouns and named entities to calculate the model. 

First, we create some functions to run the tagging.


In [ ]:
# Function to convert a document in a vector of sentences
convert_text_to_sentences_postag <- function(text, SentModel = NULL, TokModel = NULL, POSModel = NULL) {
  # Convert to NLP:String Object
  text <- NLP::as.String(text)
  # Annotate sentence boundaries
  sentenceBoundaries <- NLP::annotate(text, SentModel)
  # Select the sentences out of the original text object using the annotations
  sentences <- text[sentenceBoundaries]
  # Pass sentences to POS annotator
  sentences <- sapply(sentences, processPos, TokModel = TokModel, POSModel = POSModel)
  document <- paste(sentences["POStagged", ], collapse = " ")
  # return the documents
  return(document)
}

processPos <- function(sentence, TokModel = NULL, POSModel = NULL) {
    sentence <- as.String(sentence)
    # Add sentence annotation
    a1 <- Annotation(1L, "sentence", 1L, nchar(sentence))
    # Add token annotation
    a2 <- NLP::annotate(sentence, TokModel, a1)
    # Add POS annotation
    a3 <- NLP::annotate(sentence, POSModel, a2)
    # Select words from annotation list
    a3w <- a3[a3$type == "word"]
    # Select POS tags from annotation list
    POStags <- unlist(lapply(a3w$features, `[[`, "POS"))
    # Concat word+/+tag for all words in the sentences
    POStagged <- paste(sprintf("%s/%s", sentence[a3w], POStags), collapse = " ")
    # return result as list
    list(POStagged = POStagged, POStags = POStags)
}

# Function to convert a vector of documents into a vector of POS-Tagged documents
postag_text_source <- function(text, ...) {
  # load models
  WTA <- Maxent_Word_Token_Annotator(model = "resources/en-token.bin")
  STA <- Maxent_Sent_Token_Annotator(model = "resources/en-sent.bin")
  PTA <- Maxent_POS_Tag_Annotator(model = "resources/en-pos-maxent.bin")
  # create txt-progress bar to follow progress of pos tagging
  pb <- txtProgressBar(min = 0, max = length(text))
  i <- 0
  # loop over documents to extract sentences and pos-tag them
  docs <- sapply(text, FUN=function(x) {
    i <<- i + 1
    setTxtProgressBar(pb, i)
    convert_text_to_sentences_postag(x, SentModel = STA, TokModel = WTA, POSModel = PTA)
  }, ...)
  close(pb)
  return(docs)
}


Now we run the tagging. This may take a while. Consider loading the precomuted results with `load("pos_result.RData")`.

In [ ]:
pos_result <- postag_text_source(textdata$jobpost)
save(pos_result, file = "pos_result.RData")

Let's check the results. All tokens in the first document should now have a Part-od-Speech-Tag in the end. This allows subsequent processing steps to distinguish, for instance, the term "state" as noun ("state/nn") from state as verb ("state/vb").

In [ ]:
substr(pos_result[[1]], 0, 300)
textdata$POS_TEXT <- pos_result

Now, we can use this annotated text corpus to create a new DTM. From this DTM, we remove all tokens, which do not belong into the NN(S) classes for nouns, or NNP(S) classes for proper nouns. Finally, we compute a new topic model.

In [ ]:
# Create corpus object
data_corpus_pos <- corpus(textdata$POS_TEXT, docnames = textdata$d_id)
 

data_dfm_entries_pos <- data_corpus_pos %>% tokens(what = "fasterword") %>% tokens_tolower() %>%
  tokens_ngrams(1) %>% dfm() 


data_dfm_entries_sub_pos <- data_dfm_entries_pos %>% dfm_trim(min_docfreq = 5) 

colnames(data_dfm_entries_sub_pos) <- colnames(data_dfm_entries_sub_pos) %>% stringi::stri_replace_all_regex("[^_/a-z]", "") 

data_dfm_entries_sub_pos <- dfm_compress(data_dfm_entries_sub_pos, "features")

data_dfm_entries_sub_pos <- 
  data_dfm_entries_sub_pos[rowSums(data_dfm_entries_sub_pos) >=10, nchar(colnames(data_dfm_entries_sub_pos)) > 1]

DTM_pos <- data_dfm_entries_sub_pos

# Remove unwanted pos-tags by using a regex query to the DTM
idx <- grep(colnames(DTM_pos), perl = T, pattern = "(/nns?|/nnps?)$")

DTM_pos <- DTM_pos[, idx]

#This needs to be an extra step in order to work properly
DTM_pos <- DTM_pos[rowSums(DTM_pos) > 0, ]

# number of topics
K <- 30

# compute the LDA model, inference via 500 iterations of Gibbs sampling
topicModel_pos <- LDA(DTM_pos, K, method = "Gibbs", control = list(iter = 500, verbose = 100, alpha = 0.1, estimate.beta = TRUE))
top5termsPerTopicScore <- top.topic.words(posterior(topicModel_pos)$terms, num.words = 5, by.score = T)
print(top5termsPerTopicScore)


As you can see, the topic terms consist only (proper) nouns and might be easier to interpret.

## References

1. Blei, D., Ng, A., Jordan, M.: Latent dirichlet allocation. The Journal of Machine Learning Research. 3, 993–1022 (2003).

2. Hornik, K., Grün, B.: Topicmodels: An R package for fitting topic models. Journal of Statistical Software. 40, 1–30 (2011).

3. Chang, J., Chang, M.J.: Package “lda”. Citeseer (2010).

4. Sievert, C., Shirley, K.E.: LDAvis: A method for visualizing and interpreting topics. In: Proceedings of the workshop on interactive language learning, visualization, and interfaces. pp. 63–70 (2014).